In [2]:
"""
Automatic speech recognition and speaker identity
"""
from __future__ import division, print_function, absolute_import
import os
import random
import numpy as np
import sklearn.preprocessing as prep
import librosa
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Input,Dense, Dropout
from tensorflow.keras.layers import Embedding, Bidirectional,Conv1D,GlobalMaxPooling1D
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras import utils,losses
print("tf=",tf.__version__)
print("keras=",keras.__version__)

tf= 2.3.0
keras= 2.4.0


In [4]:
print("Imports doned")
def wav2mfcc(file_path, max_pad_len=30):
    wave, sampleRate = librosa.load(file_path, mono=True, sr=None)
    wave=prep.maxabs_scale(wave) #Normalize amplitude to +-1
    mfcc = librosa.feature.mfcc(wave, sr=sampleRate).transpose()
    #print(sampleRate,"wave=",wave.shape,"mfcc=",mfcc.shape,"file=",file_path)
    #print(mfcc[1,:5])
    dif=max_pad_len-mfcc.shape[0]
    #padding sample
    if dif>0:
        #fill holes
        #print("filling holes")
        for i in range(dif):
            mfcc=np.vstack([mfcc,np.zeros(mfcc.shape[1])])
    else:
        #take away excess
        #print("excess")
        mfcc=mfcc[:max_pad_len]
    #print(mfcc.shape)
    #print(mfcc[199,:5])
    return mfcc

#PATH_DATA="/home/francisco/nas/datasets/sonido/voz/BBDD_Smith-Magenis_Relabeling/asr_sms_21070201"
PATH_DATA="/home/spika/datasets/sonido/voz/asr_sms_m_21070801"
# load wav files with this format speaker_word_number.wav
class EasyASR(object):
    def __init__(self, filePath=PATH_DATA):
        '''
        Constructor
        '''
        self.max_pad_len=44
        self.nval=15
        basepath=filePath
        self.mfccs_n=[]
        self.speakers_n=[]
        self.diseases_n=[]
        self.ages_n=[]
        self.mfccs_d=[]
        self.speakers_d=[]
        self.diseases_d=[]
        self.ages_d=[]
        fNames=os.listdir(basepath)
        print("fNames=",len(fNames))
        random.shuffle(fNames)
        for i,fn in enumerate(fNames):
            #file name format speaker_word_number.wav
            n,_=fn.split(".")#take away extension .wav
            #decode file name format speaker-desease-age-number.wav
            #print("n=",n)
            speaker,disease,age,number=n.split("-")
            age=int(age)
            if disease!="SMS": continue
            mfccs=wav2mfcc(basepath+'/{}.wav'.format(n),self.max_pad_len)
            self.mfccs_d.append(mfccs)
            self.speakers_d.append(speaker)
            self.diseases_d.append(disease)
            self.ages_d.append(age)
        nSMS=len(self.mfccs_d)
        print("nSMS=",nSMS)
        random.shuffle(fNames)
        for i,fn in enumerate(fNames):
            #file name format speaker_word_number.wav
            n,_=fn.split(".")#take away extension .wav
            #decode file name format speaker-desease-age-number.wav
            #print("n=",n)
            speaker,disease,age,number=n.split("-")
            age=int(age)
            if disease!="N": continue
            mfccs=wav2mfcc(basepath+'/{}.wav'.format(n),self.max_pad_len)
            self.mfccs_n.append(mfccs)
            self.speakers_n.append(speaker)
            self.diseases_n.append(disease)
            self.ages_n.append(age)
            idxSMS=random.randint(self.nval,nSMS)
            self.mfccs_d.append(self.mfccs_d[idxSMS])
            self.speakers_d.append(self.speakers_d[idxSMS])
            self.diseases_d.append(self.diseases_d[idxSMS])
            self.ages_d.append(self.ages_d[idxSMS])

        self.mfccs_val   =self.mfccs_d   [:self.nval]+self.mfccs_n[:self.nval]
        self.speakers_val=self.speakers_d[:self.nval]+self.speakers_n[:self.nval]
        self.diseases_val=self.diseases_d[:self.nval]+self.diseases_n[:self.nval]
        self.ages_val    =self.ages_d    [:self.nval]+self.ages_n[:self.nval]

        self.mfccs   =self.mfccs_d   [nSMS+self.nval:]+self.mfccs_n[self.nval:]
        self.speakers=self.speakers_d[nSMS+self.nval:]+self.speakers_n[self.nval:]
        self.diseases=self.diseases_d[nSMS+self.nval:]+self.diseases_n[self.nval:]
        self.ages    =self.ages_d    [nSMS+self.nval:]+self.ages_n[self.nval:]

        self.leSpeakers=prep.LabelEncoder()
        self.leSpeakers.fit(self.speakers+self.speakers_val)
        self.iSpeakerLabels=self.leSpeakers.transform(self.speakers)
        self.iSpeakerLabels_val=self.leSpeakers.transform(self.speakers_val)
        self.leDiseases=prep.LabelEncoder()
        self.leDiseases.fit(self.diseases+self.diseases_val)
        self.iDiseaseLabels=self.leDiseases.transform(self.diseases)
        self.iDiseaseLabels_val=self.leDiseases.transform(self.diseases_val)
        self.leAges=prep.LabelEncoder()
        self.leAges.fit(self.ages+self.ages_val)
        self.iAgeLabels=self.leAges.transform(self.ages)
        self.iAgeLabels_val=self.leAges.transform(self.ages_val)
        print("Samples=",len(self.diseases))
        print("SMS=",self.diseases.count('SMS'))
        print("N   =",self.diseases.count('N'))
        print(set(self.diseases))
        print(set(self.ages))
        print(set(self.speakers))

ea=EasyASR()
print(ea.leDiseases)

Imports doned
fNames= 631
nSMS= 78
Samples= 1076
SMS= 538
N   = 538
{'N', 'SMS'}
{33, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 15, 16, 17, 18, 21, 22}
{'CMI', 'MPJ', 'PCR', 'RZI', 'PMP', 'LEY', 'PRA', 'SSA', 'PLW', 'ZRA', 'RAA', 'HSI', 'RMV', 'AMA', 'CAD', 'FEA', 'CII', 'ASC', 'HPP', 'ERJ', 'SHA', 'MAJ', 'JVB', 'JPI', 'MLM', 'GRR', '2JA', 'ICJ', 'KCR', 'MAP', 'QMA', 'ORD', 'GBR', 'NVA', 'SPI', 'ACI', 'AAJ', 'ACR', 'DAP', 'AAI', 'PPM', 'KHS', 'PLA', 'HGH', 'RMD', 'MRR', 'LAR', 'LJA', 'FAF', 'BBR', 'ROD', 'PDI', 'VFH', 'LMD', 'MOA', 'TBD', 'IMJ', 'DMD', 'GDT', 'FGI', 'FRM', 'TVD', 'MAA', 'ENF', 'PLD', 'RRR', 'VCA', 'VTR', 'DHD', 'SPD', 'CRO', 'PRV', 'FGJ', 'VJA', 'FCD', 'FLS', 'TJI', 'RGA', 'PMD', 'MMA', 'EMA', 'RCC', 'PGM', 'CSR', 'JCL', 'MMS', 'HXA', 'AJA', 'PFD', 'FFR', 'RPC', 'LAM', 'LMA', 'NVD', 'PAJ', 'MYA', 'RDA', 'BDC', 'MRA', 'NLJ', 'IMA', 'API', '11C', 'WMJ', 'RMI', 'AIE', 'DMH', 'ACD', 'LRA', 'BFA', 'VSY', 'CPD', 'RMS', 'VVA', 'MCJ', 'CRD', 'DGR', 'JMA', 'CAR', 'BHH', 'GMM', 'VGR', 'X

In [5]:
trainX, trainYdiseases, trainYspeakers, trainYages = np.array(ea.mfccs),np.array(ea.iDiseaseLabels),np.array(ea.iSpeakerLabels),np.array(ea.iAgeLabels)
#testX, testY = test
valX, valYdiseases, valYspeakers, valYages = np.array(ea.mfccs_val),np.array(ea.iDiseaseLabels_val),np.array(ea.iSpeakerLabels_val),np.array(ea.iAgeLabels_val)

maxlen=trainX.shape[1]
lstm_hidden_units=128
input_dim=trainX.shape[2]
outputDiseases_dim=len(ea.leDiseases.classes_)
outputSpeakers_dim=len(ea.leSpeakers.classes_)
outputAges_dim=len(ea.leAges.classes_)

print(outputDiseases_dim,outputSpeakers_dim,outputAges_dim)

# Converting labels to binary vectors
trainYdiseases_cat = utils.to_categorical(trainYdiseases,outputDiseases_dim)
trainYspeakers_cat = utils.to_categorical(trainYspeakers,outputSpeakers_dim)
trainYages_cat     = utils.to_categorical(trainYages    ,outputAges_dim)
print (trainX.shape,trainYdiseases_cat.shape,trainYspeakers_cat.shape,trainYages_cat.shape)

# Converting labels to binary vectors
valYdiseases_cat = utils.to_categorical(valYdiseases,outputDiseases_dim)
valYspeakers_cat = utils.to_categorical(valYspeakers,outputSpeakers_dim)
valYages_cat     = utils.to_categorical(valYages    ,outputAges_dim)
print (valX.shape,valYdiseases_cat.shape,valYspeakers_cat.shape,valYages_cat.shape)

2 126 17
(1076, 44, 20) (1076, 2) (1076, 126) (1076, 17)
(30, 44, 20) (30, 2) (30, 126) (30, 17)


In [6]:
# Network building
def build_model_asr_conv1d(input_dim,outputDiseases_dim,outputSpeakers_dim,outputAges_dim,hiden_dim=128):
    input = Input(shape=(ea.max_pad_len,input_dim))
    print("input=",input.shape)

    # Conv1D + global max pooling
    x = Conv1D(hiden_dim, 3, padding="valid", activation="relu", strides=1)(input)
    print(x.shape)
    x = Dropout(0.5)(x)
    x = Conv1D(hiden_dim, 3, padding="valid", activation="relu", strides=1)(x)
    print(x.shape)
    x = Dropout(0.5)(x)
    x = Conv1D(hiden_dim, 3, padding="valid", activation="relu", strides=1)(x)
    print(x.shape)
    #x = Dropout(0.5)(x)
    #x = Conv1D(hiden_dim, 3, padding="valid", activation="relu", strides=1)(x)
    x = GlobalMaxPooling1D()(x)
    print(x.shape)

    # We add a vanilla hidden layer:
    x = Dense(hiden_dim, activation="relu")(x)
    x = Dropout(0.5)(x)
    x = Dense(hiden_dim, activation="relu")(x)
    x = Dropout(0.5)(x)
    wd= Dense(hiden_dim, activation="relu")(x)
    sd= Dense(hiden_dim, activation="relu")(x)
    ad= Dense(hiden_dim, activation="relu")(x)
    # from: https://www.tensorflow.org/guide/keras/train_and_evaluate
    #       Passing data to multi-input, multi-output models
    prediction_diseases = Dense(outputDiseases_dim, activation="softmax", name="disease")(wd)
    prediction_speakers = Dense(outputSpeakers_dim, activation="softmax", name="speaker")(sd)
    prediction_ages     = Dense(outputAges_dim    , activation="softmax", name="age"    )(ad)

    model = Model(input, [prediction_diseases,prediction_speakers,prediction_ages])
    return model

In [7]:
model=build_model_asr_conv1d(input_dim,outputDiseases_dim,outputSpeakers_dim,outputAges_dim)
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

input= (None, 44, 20)
(None, 42, 128)
(None, 40, 128)
(None, 38, 128)
(None, 128)
Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 44, 20)]     0                                            
__________________________________________________________________________________________________
conv1d (Conv1D)                 (None, 42, 128)      7808        input_1[0][0]                    
__________________________________________________________________________________________________
dropout (Dropout)               (None, 42, 128)      0           conv1d[0][0]                     
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 40, 128)      49280       dropout[0][0]                    
_____

In [8]:
# Training
history=model.fit(trainX, [trainYdiseases_cat,trainYspeakers_cat,trainYages_cat], batch_size=100,epochs=150,validation_data=(valX, [valYdiseases_cat,valYspeakers_cat,valYages_cat]))

Epoch 1/150
11/11 [==============================] - 1s 79ms/step - loss: 82.2579 - disease_loss: 9.1618 - speaker_loss: 39.4772 - age_loss: 33.6189 - disease_accuracy: 0.4991 - speaker_accuracy: 0.0074 - age_accuracy: 0.0632 - val_loss: 8.2836 - val_disease_loss: 0.6740 - val_speaker_loss: 4.7600 - val_age_loss: 2.8496 - val_disease_accuracy: 0.6000 - val_speaker_accuracy: 0.0000e+00 - val_age_accuracy: 0.0333
Epoch 2/150
11/11 [==============================] - 0s 41ms/step - loss: 9.9737 - disease_loss: 0.9539 - speaker_loss: 5.3077 - age_loss: 3.7121 - disease_accuracy: 0.5232 - speaker_accuracy: 0.0279 - age_accuracy: 0.0920 - val_loss: 8.3090 - val_disease_loss: 0.6828 - val_speaker_loss: 4.7802 - val_age_loss: 2.8460 - val_disease_accuracy: 0.5333 - val_speaker_accuracy: 0.0333 - val_age_accuracy: 0.0667
Epoch 3/150
11/11 [==============================] - 0s 37ms/step - loss: 8.2247 - disease_loss: 0.6930 - speaker_loss: 4.7174 - age_loss: 2.8143 - disease_accuracy: 0.5530 - sp

In [9]:
idxSample=288
print(trainX[idxSample:idxSample+1,:,:].shape)
predDisease,predSpeaker,predAge=model.predict(trainX[idxSample:idxSample+1,:,:])
print(predDisease,predAge)
y_predDisease= np.argmax(predDisease)
y_predSpeaker= np.argmax(predSpeaker)
y_predAge    = np.argmax(predAge)
print(y_predDisease,"=",np.max(predDisease))
print(y_predSpeaker,"=",np.max(predSpeaker))
print(y_predAge,"="    ,np.max(predAge))
disease=ea.leDiseases.inverse_transform([y_predDisease])
speaker=ea.leSpeakers.inverse_transform([y_predSpeaker])
age    =    ea.leAges.inverse_transform([y_predAge])
print("Predictions:")
print(speaker[0]," disease is \'",disease[0],"\' with age=",age[0])
print("Groundtruth:")
print("speaker=",ea.speakers[idxSample])
print("disease=",ea.diseases[idxSample])
print("ages   =",ea.ages[idxSample])
print()
print(ea.leDiseases.classes_)
print(ea.leSpeakers.classes_)
print(ea.leAges.classes_)

(1, 44, 20)
[[7.7334994e-10 1.0000000e+00]] [[2.2096607e-08 2.3287163e-07 2.1149615e-08 2.1587786e-08 2.3012623e-08
  6.6254739e-07 1.4418322e-09 2.3233169e-09 9.9911541e-01 8.4312854e-04
  1.8065012e-07 1.0874578e-05 4.0475825e-06 9.1631127e-06 1.7023757e-07
  1.5937923e-05 1.0141547e-07]]
1 = 1.0
41 = 0.9957488
8 = 0.9991154
Predictions:
GBR  disease is ' SMS ' with age= 11
Groundtruth:
speaker= GBR
disease= SMS
ages   = 11

['N' 'SMS']
['11C' '2JA' 'AAI' 'AAJ' 'ACD' 'ACI' 'ACR' 'AIE' 'AJA' 'AMA' 'API' 'ASC'
 'BBR' 'BDC' 'BFA' 'BHH' 'CAD' 'CAR' 'CII' 'CMI' 'CPD' 'CRD' 'CRO' 'CSR'
 'DAP' 'DGR' 'DHD' 'DMD' 'DMH' 'EMA' 'ENF' 'ERJ' 'FAF' 'FCD' 'FEA' 'FFR'
 'FGI' 'FGJ' 'FLS' 'FMI' 'FRM' 'GBR' 'GDT' 'GMM' 'GRR' 'HGH' 'HPP' 'HSI'
 'HXA' 'ICJ' 'IMA' 'IMJ' 'JCL' 'JMA' 'JPI' 'JVB' 'KCR' 'KHS' 'LAM' 'LAR'
 'LEY' 'LJA' 'LMA' 'LMD' 'LRA' 'MAA' 'MAJ' 'MAP' 'MCJ' 'MLM' 'MMA' 'MMS'
 'MOA' 'MPJ' 'MRA' 'MRR' 'MYA' 'NLJ' 'NVA' 'NVD' 'ORD' 'PAJ' 'PCR' 'PDI'
 'PFD' 'PGM' 'PLA' 'PLD' 'PLW' 'PMD' 'PMP' 'PP

In [19]:
idxSample=20
print(valX[idxSample:idxSample+1,:,:].shape)
predDisease,predSpeaker,predAge=model.predict(valX[idxSample:idxSample+1,:,:])
print(predDisease,predAge)
y_predDisease= np.argmax(predDisease)
y_predSpeaker= np.argmax(predSpeaker)
y_predAge    = np.argmax(predAge)
print(y_predDisease,"=",np.max(predDisease))
print(y_predSpeaker,"=",np.max(predSpeaker))
print(y_predAge,"="    ,np.max(predAge))
disease=ea.leDiseases.inverse_transform([y_predDisease])
speaker=ea.leSpeakers.inverse_transform([y_predSpeaker])
age    =    ea.leAges.inverse_transform([y_predAge])
print("Predictions:")
print(speaker[0]," disease is \'",disease[0],"\' with age=",age[0])
print("Groundtruth:")
print("speaker=",ea.speakers_val[idxSample])
print("disease=",ea.diseases_val[idxSample])
print("ages   =",ea.ages_val[idxSample])
print()
print(ea.leDiseases.classes_)
#print(ea.leSpeakers.classes_)
print(ea.leAges.classes_)

(1, 44, 20)
[[0.9973008  0.00269924]] [[0.01081864 0.0265422  0.16573304 0.21953647 0.27376738 0.06484093
  0.12652454 0.03026626 0.07430733 0.00168311 0.0009199  0.00112538
  0.0008512  0.00087012 0.00102026 0.00046368 0.00072955]]
0 = 0.9973008
65 = 0.03295302
4 = 0.27376738
Predictions:
MAA  disease is ' N ' with age= 7
Groundtruth:
speaker= RAA
disease= N
ages   = 6

['N' 'SMS']
[ 3  4  5  6  7  8  9 10 11 13 15 16 17 18 21 22 33]
